[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rndsrc/2024_nobel-phys_hs/blob/main/Hopfield.ipynb)

# Exploring the 2024 Nobel Prize in Physics

## 0. Setup (5 min)

### Access the Notebook

To start working with this notebook, open it using one of the following methods:

- [**GitHub Codespaces**](https://github.com/features/codespaces): If you are using a GitHub repository, you can launch a Codespace, which provides an online VS Code environment with Jupyter support.
- [**Google Colab**](https://colab.research.google.com/): If you prefer a cloud-based Python environment, open this notebook in Google Colab.
- **Local Jupyter Lab:** If you have Jupyter installed on your machine, run the following command in your terminal:
  ```sh
  jupyter lab
  ```

### Quick Tour of Jupyter Notebook

Jupyter Notebook consists of **cells** that can contain either **Markdown** (formatted text) or **Python code**.

- **Markdown Cells:** Used for documentation and instructions.
  Double-click a cell to edit it, and press `Shift + Enter` to render it.
- **Code Cells:** Used to write and execute Python code.
  Click inside a cell and press `Shift + Enter` to run it.

### Example: Run Your First Python Code

Try running the following code cell to check if Python is working correctly:

In [ ]:
# Print a welcome message
def greet():
    return "Welcome to the Hopfield Network Notebook!"

print(greet())

### Installing Dependencies (if needed)

In a Jupyter Notebook, an `!` (exclamation mark) is used to execute shell (command-line) commands directly from a code cell.
This allows users to run commands as they would in a terminal, without leaving the notebook.
If running this notebook locally, ensure you have the required libraries installed:

In [ ]:
# Command

!pip install numpy matplotlib

This concludes the setup!
Now, let's dive into Hopfield networks and their connection to the Ising model.

## 1. Introduction (5 min)

### What is the 2024 Nobel Prize in Physics About?

In 2024, the **Nobel Prize in Physics** was awarded to **John J. Hopfield** and **Geoffrey E. Hinton** for their pioneering work on neural networks.
Their contributions have played a significant role in the development of modern artificial intelligence and computational models inspired by biological learning mechanisms.

- **John J. Hopfield** introduced the Hopfield network, a type of recurrent neural network that models associative memory and collective computation in neural systems.
- **Geoffrey E. Hinton** made significant advancements in deep learning, particularly in training deep neural networks through the backpropagation algorithm.

Their research has had profound implications in artificial intelligence, neuroscience, and computational physics.

### Why Do Neural Networks Matter?

Neural networks are now an essential part of many everyday technologies:

- **Facial Recognition:** Used to unlock smartphones and enhance security.
- **Chatbots:** Power customer service agents that interact with users.
- **Voice Assistants:** Siri, Alexa, and Google Assistant rely on neural networks for speech recognition.
- **Self-Driving Cars:** Autonomous vehicles process sensor data using deep learning models.

## Importance of This Discovery

Neural networks have revolutionized multiple domains, including:
- **Language Translation:** Real-time translation services are powered by deep learning models.
- **Medical Diagnosis:** AI-assisted analysis of medical images improves diagnostic accuracy.
- **Scientific Research:** AI-driven simulations enhance modeling in physics, chemistry, and biology.

## 2. The Ising Model (5 min)

### What Is the Ising Model?

To understand how neural networks operate, we first introduce the **Ising model**.

The **Ising model** is a mathematical model in statistical mechanics, introduced by Wilhelm Lenz in 1920 and solved for the one-dimensional case by his student Ernst Ising in 1925.
The model was originally used to explain ferromagnetism, where magnetic materials exhibit spontaneous magnetization due to interactions between neighboring atomic spins.

- Wilhelm Lenz conceived the model as a simplified representation of magnetic interactions in a lattice, where spins can either point "up" (+1) or "down" (-1).
- Ernst Ising solved the one-dimensional version of the model in his doctoral thesis, showing that it did not exhibit phase transitions—a result that was surprising at the time.
- Lars Onsager solved the two-dimensional version of the model in 1944, demonstrating that it undergoes a phase transition at a critical temperature, where spontaneous magnetization occurs.

Since then, the Ising model has become one of the most widely studied models in statistical physics and beyond.
Its applications extend not only to physics (such as in magnetism and lattice gases) but also to fields like biology (neural networks), computer science (optimization problems), and even sociology (modeling opinion dynamics).

The **Metropolis algorithm** (1953) was developed for Monte Carlo simulations of systems like the Ising model, enabling the study of large, complex systems by simulating their thermal fluctuations and statistical properties.
This method revolutionized computational physics and remains a powerful tool in many areas of research today.

#### Basic Idea

- Consider a **grid (lattice)** where each point represents a **spin**.
- Each spin can take one of two states: **up (+1) or down (-1)**.
- Spins interact with their neighbors, aiming to align in a way that minimizes system energy.
- The system's **energy parameter** determines the likelihood of spins flipping, making temperature a crucial factor in system behavior.

#### Interaction and Energy Minimization

- The system's total energy is determined by how well-aligned the spins are with their neighbors.
- At **high temperatures**, spins flip frequently due to thermal fluctuations, leading to disorder.
- At **low temperatures**, spins align into ordered configurations, exhibiting phase transitions similar to those in magnetic materials.
- The **Hamiltonian** (energy function) for the Ising model is:
  $$
  H = - \sum_{i,j} J_{ij} s_i s_j
  $$
  where:
  - $s_i$ represents the spin at site $i$.
  - $J_{ij}$ is the coupling strength between neighboring spins.
- The system evolves dynamically to minimize $H$, leading to patterns of stable configurations.

#### Energy Change Due to Spin Flip

When a single spin $s_i$ flips, the change in energy is calculated by comparing the energy before and after the flip.
The energy difference $\Delta E$ due to flipping the spin at site $i$ is:
$$
\Delta E = 2 s_i \sum_{j} J_{ij} s_j,
$$
where the sum is over the nearest neighbors $j$ of site $i$.
The factor of 2 arises because flipping the spin at site $i$ changes its contribution to the energy from $-s_i s_j$ to $+s_i s_j$.

### Metropolis Algorithm

To simulate the evolution of the system at a given temperature, we use the Metropolis algorithm.
This algorithm probabilistically accepts or rejects a spin flip based on the energy change $\Delta E$ and the temperature $T$.
The probability of accepting a spin flip is given by the Boltzmann factor:
$$
P(\text{flip}) =
\begin{cases}
1 & \text{if } \Delta E < 0, \\
\exp\left(-\frac{\Delta E}{k_B T}\right) & \text{if } \Delta E \geq 0,
\end{cases}
$$
where:
- $\Delta E$ is the energy change caused by the flip.
- $k_\mathrm{B}$ is the Boltzmann constant.
 $T$ is the temperature.

This allows the system to "explore" higher energy states at higher temperatures (thermal fluctuations), while favoring low-energy configurations as the system cools down.

### Why It Matters

#### Real-World Example: Magnets and Decision-Making
- In **magnetism**, atomic spins align to form magnetic domains, illustrating how interactions shape large-scale behavior.
- In **social dynamics**, human decision-making can resemble an Ising model, where individuals adjust their opinions based on peer influence.

#### Optimization
- Many complex optimization problems, including **image recognition and combinatorial search problems**, can be formulated using the Ising model framework.
- The process of **energy minimization** in the Ising model is analogous to **error minimization** in machine learning.
- Neural networks, particularly **Hopfield networks** that we will present later, use a similar principle to find stable memory states.

## 3. Hands-On Activity: Simulating a 2D Ising Model (10 min)

Here, we provide sample python codes to implement the Ising Model.

In [ ]:
# First, we import useful packages `numpy` and `matplotlib`

import numpy as np

from matplotlib import pyplot as plt
from matplotlib import image  as img

In [ ]:
# For easy comparison of simulation paramaters, we will implement the Ising model as a python class.

class IsingModel:
    """
    Ising Model Class.
    """
    
    def __init__(self, T, shape=(64,64)):
        """
        Initialize the Ising Model.
        
        Parameters:
          T: Temperature of the system.
          shape: The size of the 2D spin lattice.
        """
        self.T     = T      # Temperature
        self.shape = shape  # Lattice size
        self.state = None   # Initialize the spin state

    def random(self, seed=None):
        """
        Initialize the spin lattice randomly with +1 and -1.
        """
        if seed is not None:
            np.random.seed(seed)  # Ensure reproducibility
        self.state = np.random.choice([-1,1], size=self.shape)  # Random spins

    @staticmethod
    def dE(state, i, j):
        """
        Calculate the change in energy when flipping a spin at (i, j).
        """
        I, J = state.shape  # Get lattice dimensions
        spin = state[i, j]  # Current spin value
        # Neighboring spins (periodic boundary conditions)
        neighbors = state[(i+1)%I, j] + state[(i-1)%I, j] + state[i, (j+1)%J] + state[i, (j-1)%J]
        return 2 * spin * neighbors  # Energy difference due to spin flip

    @staticmethod
    def flip(state, i, j, T):
        """
        Decide whether to flip a spin at (i, j) based on the Metropolis algorithm.
        """
        dE = IsingModel.dE(state, i, j)  # Compute energy change
        if dE < 0:
            return True  # Always accept flip if energy decreases
        else:
            return np.random.rand() < np.exp(-dE / T)  # Accept flip with Boltzmann probability

    @staticmethod
    def step(state, i, j, T):
        """
        Perform a single spin flip step if accepted.
        """
        if IsingModel.flip(state, i, j, T):
            state[i, j] *= -1  # Flip the spin

    def run(self, N):
        """
        Run the Monte Carlo simulation for N steps.
        """
        for n in range(N):
            i = np.random.randint(0, self.shape[0])  # Random row index
            j = np.random.randint(0, self.shape[1])  # Random column index
            self.step(self.state, i, j, self.T)      # Attempt to flip the spin

In [ ]:
# Setup an Ising Model
I = IsingModel(1)  # Initialize the Ising model at a given temperature
I.random()         # Set up a random spin configuration
I.run(64*64*100)   # Run the simulation for a certain number of steps

# Display the final state of the spin lattice
plt.imshow(I.state, cmap='gray')
plt.title("Final Spin Configuration")
plt.show()

In [ ]:
# HANDS-ON: instead of just visualizing the final output, implement a loop to create a movie

pass

In [ ]:
# HANDS-ON: plot magnetization, i.e., sum(state) / state.size, as a function of step

pass

In [ ]:
# HANDS-ON: plot magnetization for different temperature

pass

## 4. Hopfield Networks (10 min)

### Memory Machines

The **Hopfield network**, introduced by **John Hopfield in 1982**, is a recurrent neural network that stores and recalls patterns through energy minimization.

- **Pattern Storage and Recall:** The network can store multiple patterns as stable configurations.
- **Robustness to Noise:** If an input is noisy or incomplete, the network still retrieves the correct stored pattern.
- **Content-Addressable Memory:** Unlike conventional memory, which retrieves data using addresses, Hopfield networks retrieve patterns based on similarity.

### Energy Landscape

The Hopfield network shares deep similarities with the Ising model:
- **Neurons replace spins**, taking binary values $+1$ or $-1$.
- **Connections between neurons** correspond to interactions between spins.
- **The system evolves toward stable, low-energy states**, just like in the Ising model.
- **Both exhibit emergent global order** despite being governed by local interactions.

The energy function of the Hopfield network is given by:
$$
E = - \sum_{i,j} W_{ij} \sigma_i \sigma_j,
$$
where $W_{ij}$ represents synaptic weights and $\sigma_i$ is the neuron state.

When a neuron updates, its **energy change** follows:
$$
\Delta E = 2 \sigma_i \sum_j W_{ij} \sigma_j.
$$

### Why It Matters

The Hopfield network played a **foundational role in artificial intelligence** by introducing energy-based models:
- Inspired later developments such as **Boltzmann machines** and **Deep Learning architectures**.
- Showed how memory and pattern recognition could emerge from simple neuron interactions.
- Provided insights into **biological neural networks**, helping us understand how the brain processes and retrieves information.

### Hebbian Learning Rule

The weights in a Hopfield network are learned using the Hebbian learning rule, which strengthens the connections between neurons that are activated together.
It is given by:
$$
W_{ij} = \frac{1}{P} \sum_p \sigma_i^{(p)} \sigma_j^{(p)}.
$$
Where:
* $N$ is the number of neurons.
* $P$ is the number of patterns.
* $\sigma_i^{(p)}$ is the state of neuron $i$ in a pattern $p$.

This makes Hopfield networks **powerful for associative memory**, allowing pattern retrieval even with noise.

## 5. Hands-On Activity: Building a Hopfield Network (10 min)

In this section, we will implement a **Hopfield Network** and train it to recognize patterns.

In [ ]:
# Define the Hopfield Network Class

class HopfieldNetwork:
    """
    Hopfield Network Class.
    """
    
    def __init__(self, shape=(64, 64)):
        """
        Initialize the Hopfield network.
        
        Parameters:
          shape: The size of the 2D pattern.
        """
        self.shape = shape
        self.W     = np.zeros((shape[0] * shape[1], shape[0] * shape[1]))
        self.state = None

    def random(self, seed=None):
        """
        Initialize the state of the network with random values of +1 and -1.
        """
        if seed is not None:
            np.random.seed(seed)
        self.state = np.random.choice([-1, 1], size=self.shape)

    def train(self, patterns):
        """
        Train the network using the Hebbian learning rule.
        
        Parameters:
          patterns: A list of binary patterns (numpy arrays of -1 and 1) to be stored.
        """
        for p in patterns:
            assert p.shape == self.shape
            p_flat = p.flatten()
            self.W += np.outer(p_flat, p_flat)
        np.fill_diagonal(self.W, 0)  # Ensure no neuron connects to itself
        self.W /= len(patterns)      # Normalize by the number of patterns

    @staticmethod
    def step(state, i, j, W):
        """
        Update a single neuron using the network's weight matrix.
        """
        index = i * state.shape[1] + j
        state[i, j] = np.sign(np.dot(W[index], state.flatten()))

    def run(self, N):
        """
        Evolve the network for N iterations.
        """
        for _ in range(N):
            i = np.random.randint(0, self.shape[0])
            j = np.random.randint(0, self.shape[1])
            self.step(self.state, i, j, self.W)

In [ ]:
# Define an image loader that read and convert an image to binary (-1, 1) using `matplotlib`

def load_image(filename):
    im = img.imread(filename)
    if im.ndim == 3:
        im = np.mean(im, axis=-1)   # Convert to grayscale if needed
    im = np.where(im < 128, -1, 1)  # Convert grayscale to binary (-1, 1)
    return im

In [ ]:
# Load images and display them

im1 = load_image("images/AI.jpg")
im2 = load_image("images/cloud.jpg")

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(im1, cmap='gray')
ax0.set_title('Pattern "AI"')
ax1.imshow(im2, cmap='gray')
ax1.set_title('Pattern "cloud"')
plt.show()

In [ ]:
# Train the Hopfield Network

h = HopfieldNetwork()
h.train([im1, im2])

In [ ]:
# Test the Network

h.random()      # Initialize a random state 
h.run(64*64*8)  # ... and evolve it to retrieve a pattern

In [ ]:
# Display the retrieved pattern

plt.imshow(h.state, cmap='gray')
plt.title("Retrieved Pattern")
plt.show()

In [ ]:
# HANDS-ON: instead of just visualizing the final output, implement a loop to create a movie

pass

In [ ]:
# HANDS-ON: instead of starting randomly, introduce structured noise to observe how the network behaves.

pass

In [ ]:
# HANDS-ON: load additional images and test the network's recall capacity

pass

## 6. Discussion (5 min)
- **Real-World Applications:**
  - How neural networks are used in AI, data analysis, and more.
- **Ethical Considerations:**
  - Discuss fairness, biases, and the impact of AI on society.
- **Closing Questions:**
  - How do physics and AI work together in models like the Ising model and Hopfield Networks?
  - Encourage further exploration with tools like Jupyter Notebooks.